In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import naive_bayes
from sklearn import preprocessing
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc 

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.family']='sans-serif'

In [2]:
train_data  = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

train_data['is_train'] = 1
test_data['is_train'] = 0
test_id = test_data.Id
train_y = train_data[['Category']]
train_data = train_data.drop(['Category','Descript','Resolution'],axis = 1)
test_data = test_data.drop('Id',axis = 1)
combine_data = pd.concat([train_data,test_data],axis=0,ignore_index=True)


del train_data,test_data
gc.collect()

51

In [3]:
#完成标签编码
encoder = preprocessing.LabelEncoder() 
train_y = encoder.fit_transform(train_y.Category)

In [4]:
#对特征进行处理
def extract_time_feature(data):
    print(u'开始提取时间特征')
    time_feature = pd.to_datetime(data.Dates)
    data['year'] = time_feature.dt.year
    data['month'] = time_feature.dt.month
    data['day'] = time_feature.dt.day
    data['hour'] = time_feature.dt.hour
    data['minute'] = time_feature.dt.minute
    data['second'] = time_feature.dt.second
    print(u'提取时间特征完成')

extract_time_feature(combine_data)
combine_data = combine_data.drop('Dates',axis = 1)

开始提取时间特征
提取时间特征完成


In [6]:
#直接对所有特征都进行label encoder
feature = ['Address','DayOfWeek','PdDistrict']
for f in feature:
    feature_encoder = preprocessing.LabelEncoder()
    combine_data[f] = feature_encoder.fit_transform(combine_data[f])

In [10]:
#去掉不用的项
combine_data = combine_data.drop(['X','Y'],axis=1)

ValueError: labels ['X' 'Y'] not contained in axis

In [11]:
train_X = combine_data.loc[combine_data.is_train==1,:].drop('is_train',axis=1)
test_X = combine_data.loc[combine_data.is_train==0,:].drop('is_train',axis=1)

In [16]:
#下面搭建贝叶斯模型
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

tr_X ,val_X , tr_y ,val_y = train_test_split(train_X,train_y,train_size = .75)

nb = naive_bayes.BernoulliNB()
nb.fit(tr_X,tr_y)
cv_predict = np.array(nb.predict_proba(val_X))
print(u'朴素贝叶斯的log损失为 %f'%(log_loss(val_y,cv_predict)))


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


朴素贝叶斯的log损失为 2.612564


In [22]:
submission = nb.predict_proba(test_X)
submission = pd.DataFrame(submission)
submission.columns = encoder.inverse_transform(submission.columns)

In [38]:
predict_sub = pd.concat([test_id,submission],axis= 1 )

In [41]:
predict_sub.astype(np.float16)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.000000,0.005100,0.120972,0.000145,0.000614,0.033936,0.003195,0.002264,0.068298,0.003704,...,0.000125,0.005409,0.000435,0.044098,1.370907e-05,0.008652,0.048340,0.050446,0.064331,0.022552
1,1.000000,0.005100,0.120972,0.000145,0.000614,0.033936,0.003195,0.002264,0.068298,0.003704,...,0.000125,0.005409,0.000435,0.044098,1.370907e-05,0.008652,0.048340,0.050446,0.064331,0.022552
2,2.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
3,3.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
4,4.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
5,5.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
6,6.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
7,7.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
8,8.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192
9,9.000000,0.001636,0.094482,0.000226,0.000291,0.032715,0.006668,0.003344,0.087524,0.006741,...,0.000081,0.006485,0.000643,0.035706,2.086163e-06,0.010574,0.040588,0.037231,0.065735,0.011192


In [42]:
predict_sub.to_csv('data/submission.csv',index=False ,sep=',')

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.xticks(size = 10 , rotation = -45)
plt.title(u'犯罪总数统计')
plt.xlabel(u'犯罪类型')
plt.ylabel(u'犯罪种类')

sns.countplot('Category',data =train_data)

In [ ]:
#看看有无缺失值
print(train_data.isnull().sum())
print(test_data.isnull().sum())

In [30]:
submission = pd.DataFrame(submission)

In [34]:
submission.columns = encoder.inverse_transform(submission.columns)

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
submission

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.005100,0.120997,0.000145,0.000614,0.033946,0.003194,0.002264,0.068295,0.003705,0.000712,...,0.000125,0.005408,0.000435,0.044092,1.369685e-05,0.008654,0.048331,0.050432,0.064339,0.022554
1,0.005100,0.120997,0.000145,0.000614,0.033946,0.003194,0.002264,0.068295,0.003705,0.000712,...,0.000125,0.005408,0.000435,0.044092,1.369685e-05,0.008654,0.048331,0.050432,0.064339,0.022554
2,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
3,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
4,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
5,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
6,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
7,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
8,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
9,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,0.000775,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195


,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.005100,0.120997,0.000145,0.000614,0.033946,0.003194,0.002264,0.068295,0.003705,...,0.000125,0.005408,0.000435,0.044092,1.369685e-05,0.008654,0.048331,0.050432,0.064339,0.022554
1,1,0.005100,0.120997,0.000145,0.000614,0.033946,0.003194,0.002264,0.068295,0.003705,...,0.000125,0.005408,0.000435,0.044092,1.369685e-05,0.008654,0.048331,0.050432,0.064339,0.022554
2,2,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
3,3,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
4,4,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
5,5,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
6,6,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
7,7,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
8,8,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195
9,9,0.001635,0.094498,0.000226,0.000291,0.032719,0.006669,0.003344,0.087550,0.006742,...,0.000081,0.006483,0.000643,0.035713,2.102530e-06,0.010578,0.040581,0.037245,0.065755,0.011195


array([21, 21, 16, ..., 21, 21, 21], dtype=int64)